In [8]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchonn as onn
from torchonn.models import ONNBaseModel
import torch.optim as optim
import torchvision.transforms as transforms
import scipy.stats as stats
from copy import deepcopy
from PIL import Image, ImageFilter
import os

In [9]:
class OmniglotNShot:

    def __init__(self, root, batchsz, n_way, k_shot, k_query, imgsz):
        """
        Different from mnistNShot, the
        :param root:
        :param batchsz: task num
        :param n_way:
        :param k_shot:
        :param k_qry:
        :param imgsz:
        """

        self.resize = imgsz
        if not os.path.isfile(os.path.join(root, 'omniglot.npy')):
            # if root/data.npy does not exist, just download it
            self.x = Omniglot(root, download=True,
                              transform=transforms.Compose([lambda x: Image.open(x).convert('L'),
                                                            lambda x: x.resize((imgsz, imgsz)),
                                                            lambda x: np.reshape(x, (imgsz, imgsz, 1)),
                                                            lambda x: np.transpose(x, [2, 0, 1]),
                                                            lambda x: x/255.])
                              )

            temp = dict()  # {label:img1, img2..., 20 imgs, label2: img1, img2,... in total, 1623 label}
            for (img, label) in self.x:
                if label in temp.keys():
                    temp[label].append(img)
                else:
                    temp[label] = [img]

            self.x = []
            for label, imgs in temp.items():  # labels info deserted , each label contains 20imgs
                self.x.append(np.array(imgs))

            # as different class may have different number of imgs
            self.x = np.array(self.x).astype(np.float)  # [[20 imgs],..., 1623 classes in total]
            # each character contains 20 imgs
            print('data shape:', self.x.shape)  # [1623, 20, 84, 84, 1]
            temp = []  # Free memory
            # save all dataset into npy file.
            np.save(os.path.join(root, 'omniglot.npy'), self.x)
            print('write into omniglot.npy.')
        else:
            # if data.npy exists, just load it.
            self.x = np.load(os.path.join(root, 'omniglot.npy'))
            print('load from omniglot.npy.')

        # [1623, 20, 84, 84, 1]
        # TODO: can not shuffle here, we must keep training and test set distinct!
        self.x_train, self.x_test = self.x[:1200], self.x[1200:]

        # self.normalization()

        self.batchsz = batchsz
        self.n_cls = self.x.shape[0]  # 1623
        self.n_way = n_way  # n way
        self.k_shot = k_shot  # k shot
        self.k_query = k_query  # k query
        assert (k_shot + k_query) <=20

        # save pointer of current read batch in total cache
        self.indexes = {"train": 0, "test": 0}
        self.datasets = {"train": self.x_train, "test": self.x_test}  # original data cached
        print("DB: train", self.x_train.shape, "test", self.x_test.shape)

        self.datasets_cache = {"train": self.load_data_cache(self.datasets["train"]),  # current epoch data cached
                               "test": self.load_data_cache(self.datasets["test"])}

    def normalization(self):
        """
        Normalizes our data, to have a mean of 0 and sdt of 1
        """
        self.mean = np.mean(self.x_train)
        self.std = np.std(self.x_train)
        self.max = np.max(self.x_train)
        self.min = np.min(self.x_train)
        # print("before norm:", "mean", self.mean, "max", self.max, "min", self.min, "std", self.std)
        self.x_train = (self.x_train - self.mean) / self.std
        self.x_test = (self.x_test - self.mean) / self.std

        self.mean = np.mean(self.x_train)
        self.std = np.std(self.x_train)
        self.max = np.max(self.x_train)
        self.min = np.min(self.x_train)

    # print("after norm:", "mean", self.mean, "max", self.max, "min", self.min, "std", self.std)

    def load_data_cache(self, data_pack):
        """
        Collects several batches data for N-shot learning
        :param data_pack: [cls_num, 20, 84, 84, 1]
        :return: A list with [support_set_x, support_set_y, target_x, target_y] ready to be fed to our networks
        """
        #  take 5 way 1 shot as example: 5 * 1
        setsz = self.k_shot * self.n_way
        querysz = self.k_query * self.n_way
        data_cache = []

        # print('preload next 50 caches of batchsz of batch.')
        for sample in range(10):  # num of episodes

            x_spts, y_spts, x_qrys, y_qrys = [], [], [], []
            for i in range(self.batchsz):  # one batch means one set

                x_spt, y_spt, x_qry, y_qry = [], [], [], []
                selected_cls = np.random.choice(data_pack.shape[0], self.n_way, False)

                for j, cur_class in enumerate(selected_cls):

                    selected_img = np.random.choice(20, self.k_shot + self.k_query, False)

                    # meta-training and meta-test
                    x_spt.append(data_pack[cur_class][selected_img[:self.k_shot]])
                    x_qry.append(data_pack[cur_class][selected_img[self.k_shot:]])
                    y_spt.append([j for _ in range(self.k_shot)])
                    y_qry.append([j for _ in range(self.k_query)])

                # shuffle inside a batch
                perm = np.random.permutation(self.n_way * self.k_shot)
                x_spt = np.array(x_spt).reshape(self.n_way * self.k_shot, 1, self.resize, self.resize)[perm]
                y_spt = np.array(y_spt).reshape(self.n_way * self.k_shot)[perm]
                perm = np.random.permutation(self.n_way * self.k_query)
                x_qry = np.array(x_qry).reshape(self.n_way * self.k_query, 1, self.resize, self.resize)[perm]
                y_qry = np.array(y_qry).reshape(self.n_way * self.k_query)[perm]

                # append [sptsz, 1, 84, 84] => [b, setsz, 1, 84, 84]
                x_spts.append(x_spt)
                y_spts.append(y_spt)
                x_qrys.append(x_qry)
                y_qrys.append(y_qry)


            # [b, setsz, 1, 84, 84]
            x_spts = np.array(x_spts).astype(np.float32).reshape(self.batchsz, setsz, 1, self.resize, self.resize)
            y_spts = np.array(y_spts).astype(np.int32).reshape(self.batchsz, setsz)
            # [b, qrysz, 1, 84, 84]
            x_qrys = np.array(x_qrys).astype(np.float32).reshape(self.batchsz, querysz, 1, self.resize, self.resize)
            y_qrys = np.array(y_qrys).astype(np.int32).reshape(self.batchsz, querysz)

            data_cache.append([x_spts, y_spts, x_qrys, y_qrys])

        return data_cache

    def next(self, mode='train'):
        """
        Gets next batch from the dataset with name.
        :param mode: The name of the splitting (one of "train", "val", "test")
        :return:
        """
        # update cache if indexes is larger cached num
        if self.indexes[mode] >= len(self.datasets_cache[mode]):
            self.indexes[mode] = 0
            self.datasets_cache[mode] = self.load_data_cache(self.datasets[mode])

        next_batch = self.datasets_cache[mode][self.indexes[mode]]
        self.indexes[mode] += 1

        return next_batch

In [10]:
db_train = OmniglotNShot('omniglot',
                       batchsz=32,
                       n_way=5,
                       k_shot=12,
                       k_query=8,
                       imgsz=28)

load from omniglot.npy.
DB: train (1200, 20, 1, 28, 28) test (423, 20, 1, 28, 28)


In [11]:
num_epochs = 100

device = torch.device('cpu')

x_spt_ua = torch.from_numpy(np.array([0]))
y_spt_ua = torch.from_numpy(np.array([0]))
x_qry_ua = torch.from_numpy(np.array([0]))
y_qry_ua = torch.from_numpy(np.array([0]))

x_spt, y_spt, x_qry, y_qry = db_train.next()
x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).to(device), \
                             torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).to(device)

In [20]:
!python omniglot_train.py

Namespace(epoch=40000, n_way=5, k_spt=1, k_qry=15, imgsz=28, imgc=1, task_num=32, meta_lr=0.001, update_lr=0.4, update_step=5, update_step_test=10)
Meta(
  (net): Learner(
    conv2d:(ch_in:1, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:2x2, stride:1, padding:0)
    relu:(True,)
    bn:(64,)
    flatten:()
    linear:(in:64, out:5)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 64x1x3x3]
        (1): Parameter containing: [torch.float32 of size 64]
        (2): Parameter containing: [torch.float32 of size 64]
        (3): Parameter containing: [torch.float32 of size 64]
        (4): Parameter containing: [torch.float32 of size 64x64x3x3]
        (5): Parameter containing: [torch.float32 of size 64]
       

step: 250 	training acc: [0.19083333 0.805      0.82458333 0.82375    0.82791667 0.8275    ]
val_accs: [0.3804166666666667, 0.40791666666666665, 0.4175, 0.4454166666666667, 0.39208333333333334, 0.43333333333333335, 0.4558333333333333, 0.4725, 0.47375, 0.43416666666666665, 0.4483333333333333, 0.44208333333333333, 0.50875, 0.4625, 0.48375, 0.4816666666666667, 0.48583333333333334, 0.5195833333333333, 0.50125, 0.52375, 0.5404166666666667, 0.5341666666666667, 0.5225, 0.53875, 0.5333333333333333, 0.5475, 0.5508333333333333, 0.5716666666666667, 0.5308333333333334, 0.5741666666666667, 0.5829166666666666, 0.54125, 0.5691666666666667, 0.59625, 0.59125, 0.56625, 0.5741666666666667, 0.5679166666666666, 0.5920833333333333, 0.5925, 0.5791666666666667, 0.59625, 0.6233333333333333, 0.59875, 0.60875, 0.5754166666666667, 0.5991666666666666, 0.5808333333333333, 0.60375, 0.6425, 0.61625, 0.6041666666666666, 0.6229166666666667, 0.6479166666666667, 0.6595833333333333, 0.6166666666666667, 0.6591666666666667,

step: 350 	training acc: [0.17208333 0.85166667 0.86875    0.87041667 0.87166667 0.87291667]
val_accs: [0.3804166666666667, 0.40791666666666665, 0.4175, 0.4454166666666667, 0.39208333333333334, 0.43333333333333335, 0.4558333333333333, 0.4725, 0.47375, 0.43416666666666665, 0.4483333333333333, 0.44208333333333333, 0.50875, 0.4625, 0.48375, 0.4816666666666667, 0.48583333333333334, 0.5195833333333333, 0.50125, 0.52375, 0.5404166666666667, 0.5341666666666667, 0.5225, 0.53875, 0.5333333333333333, 0.5475, 0.5508333333333333, 0.5716666666666667, 0.5308333333333334, 0.5741666666666667, 0.5829166666666666, 0.54125, 0.5691666666666667, 0.59625, 0.59125, 0.56625, 0.5741666666666667, 0.5679166666666666, 0.5920833333333333, 0.5925, 0.5791666666666667, 0.59625, 0.6233333333333333, 0.59875, 0.60875, 0.5754166666666667, 0.5991666666666666, 0.5808333333333333, 0.60375, 0.6425, 0.61625, 0.6041666666666666, 0.6229166666666667, 0.6479166666666667, 0.6595833333333333, 0.6166666666666667, 0.6591666666666667,

step: 450 	training acc: [0.20916667 0.87       0.89708333 0.89916667 0.89833333 0.89916667]
val_accs: [0.3804166666666667, 0.40791666666666665, 0.4175, 0.4454166666666667, 0.39208333333333334, 0.43333333333333335, 0.4558333333333333, 0.4725, 0.47375, 0.43416666666666665, 0.4483333333333333, 0.44208333333333333, 0.50875, 0.4625, 0.48375, 0.4816666666666667, 0.48583333333333334, 0.5195833333333333, 0.50125, 0.52375, 0.5404166666666667, 0.5341666666666667, 0.5225, 0.53875, 0.5333333333333333, 0.5475, 0.5508333333333333, 0.5716666666666667, 0.5308333333333334, 0.5741666666666667, 0.5829166666666666, 0.54125, 0.5691666666666667, 0.59625, 0.59125, 0.56625, 0.5741666666666667, 0.5679166666666666, 0.5920833333333333, 0.5925, 0.5791666666666667, 0.59625, 0.6233333333333333, 0.59875, 0.60875, 0.5754166666666667, 0.5991666666666666, 0.5808333333333333, 0.60375, 0.6425, 0.61625, 0.6041666666666666, 0.6229166666666667, 0.6479166666666667, 0.6595833333333333, 0.6166666666666667, 0.6591666666666667,

Test acc: [0.2001 0.8706 0.887  0.8896 0.891  0.8916 0.8926 0.8926 0.893  0.893
 0.8936]
test_accs_list: [0.4302, 0.8936]
step: 550 	training acc: [0.17791667 0.85208333 0.87916667 0.88583333 0.88625    0.88666667]
val_accs: [0.3804166666666667, 0.40791666666666665, 0.4175, 0.4454166666666667, 0.39208333333333334, 0.43333333333333335, 0.4558333333333333, 0.4725, 0.47375, 0.43416666666666665, 0.4483333333333333, 0.44208333333333333, 0.50875, 0.4625, 0.48375, 0.4816666666666667, 0.48583333333333334, 0.5195833333333333, 0.50125, 0.52375, 0.5404166666666667, 0.5341666666666667, 0.5225, 0.53875, 0.5333333333333333, 0.5475, 0.5508333333333333, 0.5716666666666667, 0.5308333333333334, 0.5741666666666667, 0.5829166666666666, 0.54125, 0.5691666666666667, 0.59625, 0.59125, 0.56625, 0.5741666666666667, 0.5679166666666666, 0.5920833333333333, 0.5925, 0.5791666666666667, 0.59625, 0.6233333333333333, 0.59875, 0.60875, 0.5754166666666667, 0.5991666666666666, 0.5808333333333333, 0.60375, 0.6425, 0.6162

^C
Traceback (most recent call last):
  File "/Users/matthewho/Photonic_computing/MAML-Pytorch/omniglot_train.py", line 106, in <module>
    main(args)
  File "/Users/matthewho/Photonic_computing/MAML-Pytorch/omniglot_train.py", line 61, in main
    accs = maml(x_spt, y_spt, x_qry, y_qry)
  File "/Users/matthewho/Photonic_computing/photonics_env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/matthewho/Photonic_computing/MAML-Pytorch/meta.py", line 103, in forward
    logits_q = self.net(x_qry[i], fast_weights, bn_training=True)
  File "/Users/matthewho/Photonic_computing/photonics_env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/matthewho/Photonic_computing/MAML-Pytorch/learner.py", line 144, in forward
    x = F.conv2d(x, w, b, stride=param[4], padding=param[5])
KeyboardInterrupt


In [22]:
import torch

model = torch.load('omniglot_maml.pth')

In [23]:
model(x_spt, y_spt, x_qry, y_qry)

array([0.2390625 , 0.975     , 0.984375  , 0.984375  , 0.98515625,
       0.98515625])